In [1]:
import numpy as np
import pandas as pd
import ampligraph

In [2]:
ampligraph.__version__

'1.2.0'

In [3]:
df = pd.read_pickle("../Data/cleanDataset.pkl")

In [5]:
from sklearn.model_selection import train_test_split
unused, reduceddf = train_test_split(df, test_size=0.3)


,ECG,EMG,EDA,TEMP,RESP,Sujeto,label
18342198,-0.072922,-0.001968,5.282974,33.180786,1.466370,S17,1
7870939,0.139023,-0.000046,6.993103,34.949463,1.976013,S5,4
148517,-0.056534,0.000778,2.931976,34.154999,-2.503967,S13,4
7365360,-0.199127,-0.007965,5.479431,33.511169,-1.211548,S17,1
27967559,0.023483,0.004990,4.878235,34.518951,-16.859436,S13,2
...,...,...,...,...,...,...,...
4297257,-0.019592,-0.015930,5.331421,33.415985,-1.885986,S17,1
29972115,-0.023209,0.004349,7.131958,32.893768,2.793884,S3,4
25930291,-0.017395,0.002335,3.726578,35.077606,1.390076,S8,4
28303972,-0.030624,-0.014236,3.686905,34.795380,-1.704407,S5,1


In [6]:
type(reduceddf)

pandas.core.frame.DataFrame

In [9]:
reduceddf["train"] = reduceddf.Sujeto > "S14"

/home/jaime/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
reduceddf.train.value_counts()

True     6894919
False    2546262
Name: train, dtype: int64

In [16]:
df

,ECG,EMG,EDA,TEMP,RESP,Sujeto,label,train
64564,-0.014648,-0.001968,5.543137,32.567444,2.757263,S17,1,True
64565,-0.001236,-0.005539,5.542755,32.567444,2.742004,S17,1,True
64566,0.007278,0.017761,5.556107,32.561493,2.603149,S17,1,True
64567,0.009201,0.032181,5.558014,32.559998,2.749634,S17,1,True
64568,0.007141,-0.011032,5.564117,32.558502,2.760315,S17,1,True
...,...,...,...,...,...,...,...,...
4111478,-0.182602,-0.015793,0.484085,31.926239,-1.609802,S2,4,True
4111479,-0.170609,0.000687,0.473404,31.932190,-1.646423,S2,4,True
4111480,-0.160812,0.004532,0.463486,31.918823,-1.643372,S2,4,True
4111481,-0.156326,0.000595,0.459290,31.932190,-1.661682,S2,4,True


In [20]:
df["data_id"] = df.index.values.astype(str)
df["data_id"] = "Dato" + df.data_id
df["subject_id"] = df.Sujeto.values.astype(str)

In [ ]:
triples = []
for _, row in df[df["train"]].iterrows():
    # Data info
    ecg_triple = (row["ECG"], "isECGDataIn", row["data_id"])
    emg_triple = (row["EMG"], "isEMGDataIn", row["data_id"])
    eda_triple = (row["EDA"], "isEDADataIn", row["data_id"])
    temp_triple = (row["TEMP"], "isTEMPDataIn", row["data_id"])
    resp_triple = (row["RESP"], "isRESPDataIn", row["data_id"])
    
    # Stress results
    if row["label"] == 1:
        label_triple = (row["data_id"], "ShowsAStateOf", "Baseline")
    elif row["label"] == 2:
        label_triple = (row["data_id"], "FeelingOfData", "Stress")
    elif row["label"] == 3:
        label_triple = (row["data_id"], "FeelingOfData", "Amusement")
    elif row["label"] == 4:
        label_triple = (row["data_id"], "FeelingOfData", "Meditaion")
    
    suj_triple = (row["data_id"], "isProducedBy", row["subject_id"])
    
    triples.extend((ecg_triple, emg_triple, eda_triple, temp_triple, 
                    resp_triple, label_triple, suj_triple))

In [ ]:
triples_df = pd.DataFrame(triples, columns=["subject", "predicate", "object"])
triples_df[(triples_df.subject=="Dato64566") | (triples_df.object=="Dato64566")]

In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_valid = train_test_split_no_unseen(np.array(triples), test_size=10000)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

In [ ]:
from ampligraph.latent_features import ComplEx

model = ComplEx(batches_count=50,
                epochs=300,
                k=100,
                eta=20,
                optimizer='adam', 
                optimizer_params={'lr':1e-4},
                loss='multiclass_nll',
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                seed=0, 
                verbose=True)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train)

In [ ]:
from ampligraph.utils import save_model

save_model(model, model_name_path = "../Data/KGmodel.pkl")